# Analiza i Bazy Danych
## Karol Musiał, grupa 3a
### Laboratorium 4
## "Łączenie tabel, grupowanie i funkcje agregujące"

In [145]:
import main
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

#### Ponieważ dwa pierwsze polecenia nie są sprecyzowane, tzn. nie wiadomo do jakich konkretnych przypadków się odnoszą, w każdym zadaniu wybrano parametr tak, by jak najlepiej oddawał on działanie algorytmu.

### <b>1. Znajdź listę wszystkich filmów o tej samej długości.<b>

In [146]:
film_length = 48 # Wybrano filmy o długości 48

df = pd.read_sql("SELECT film.film_id, film.title, film.length FROM film " +
                 "WHERE film.length = {a}".format(a=film_length), con=connection)
df

,film_id,title,length
0,2,Ace Goldfinger,48
1,410,Heaven Freedom,48
2,753,Rush Goodfellas,48
3,575,Midsummer Groundhog,48
4,630,Notting Speakeasy,48
5,634,Odds Boogie,48
6,657,Paradise Sabrina,48
7,670,Pelican Comforts,48
8,845,Stepmom Dream,48
9,866,Sunset Racer,48


### <b>2. Znajdź wszystkich klientów mieszkających w tym samym mieście.<b>

In [147]:
city_name = 'Abu Dhabi' # Wybrano miasto Abu Dhabi

df = pd.read_sql("SELECT customer.first_name, customer.last_name, city.city FROM customer " + 
                 "INNER JOIN address ON customer.address_id = address.address_id " + 
                 "INNER JOIN city ON address.city_id = city.city_id " + 
                 "WHERE city.city = '{a}'".format(a=city_name), con=connection)
df

,first_name,last_name,city
0,Tom,Milner,Abu Dhabi


### <b>3. Oblicz średni koszt wypożyczenia wszystkich filmów.<b>

In [148]:
df = pd.read_sql("SELECT AVG(payment.amount) FROM film INNER JOIN inventory ON film.film_id = inventory.film_id " +
                 "INNER JOIN rental ON inventory.inventory_id = rental.inventory_id " + 
                 "INNER JOIN payment ON rental.rental_id = payment.rental_id", con=connection)
avg = df['avg'][0]

print("Średni koszt wypożyczenia filmu wynosi: ${a:.2f} ".format(a=avg))

Średni koszt wypożyczenia filmu wynosi: $4.20 


### <b>4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.<b>

In [149]:
df = pd.read_sql("SELECT category.name category, COUNT(film) films FROM film INNER JOIN film_category " +
                 "ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name", con=connection)
df

,category,films
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


### <b>5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.<b>

In [150]:
df = pd.read_sql("SELECT country.country, COUNT(customer) no_clients FROM customer INNER JOIN address ON " +
                 "customer.address_id = address.address_id " +
                 "INNER JOIN city ON address.city_id = city.city_id " +
                 "INNER JOIN country ON city.country_id = country.country_id GROUP BY country.country", con=connection)
df

,country,no_clients
0,Cambodia,2
1,Turkey,15
2,Germany,7
3,Madagascar,1
4,Chad,1
...,...,...
103,Sudan,2
104,United Arab Emirates,3
105,Tanzania,3
106,French Polynesia,2


### <b>6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.<b>

In [151]:
df = pd.read_sql("SELECT store.store_id, COUNT(customer) no_clients FROM store INNER JOIN customer ON " +
                 "store.store_id = customer.store_id " +
                 "GROUP BY store.store_id HAVING COUNT(customer) BETWEEN 100 AND 300", con=connection)
df

,store_id,no_clients
0,2,273


### <b>7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.<b>

In [152]:
df = pd.read_sql("SELECT customer.first_name, customer.last_name, SUM(film.length) hours FROM customer INNER JOIN rental " +
                 "ON customer.customer_id = rental.customer_id " + 
                 "INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id " +
                 "INNER JOIN film ON inventory.film_id = film.film_id GROUP BY customer.first_name, customer.last_name " +
                 "HAVING SUM(film.length) > 200", con=connection)
df

,first_name,last_name,hours
0,Scott,Shelley,2916
1,Tim,Cary,4476
2,Samuel,Marlow,2291
3,Vera,Mccoy,2275
4,Nelson,Christenson,2119
...,...,...,...
594,Erica,Matthews,2727
595,Stanley,Scroggins,3694
596,Ellen,Simpson,2977
597,Patsy,Davidson,3513


### <b>8. Oblicz średnią wartość wypożyczenia filmu.<b>

## UWAGA!
### Zadanie 8 takie samo jak Zadanie 3

In [153]:
df = pd.read_sql("SELECT AVG(payment.amount) FROM film INNER JOIN inventory ON film.film_id = inventory.film_id " +
                 "INNER JOIN rental ON inventory.inventory_id = rental.inventory_id " + 
                 "INNER JOIN payment ON rental.rental_id = payment.rental_id", con=connection)
avg = df['avg'][0]

print("Średni koszt wypożyczenia filmu wynosi: ${a:.2f} ".format(a=avg))

Średni koszt wypożyczenia filmu wynosi: $4.20 


### <b>9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.<b>

In [154]:
df = pd.read_sql("SELECT category.name category, AVG(film.length) avg_length FROM film INNER JOIN " +
                 "film_category ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name", con=connection)
df

,category,avg_length
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


### <b>10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.<b>

In [155]:
df = pd.read_sql("SELECT category.name category, MAX(film.length) max_length FROM film INNER JOIN film_category " +
                 "ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name", con=connection)
df

,category,max_length
0,Horror,181
1,Comedy,185
2,Sci-Fi,185
3,Drama,181
4,Foreign,184
5,Classics,184
6,Games,185
7,New,183
8,Travel,185
9,Music,185


### <b>11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.<b>

In [156]:
df = pd.read_sql("SELECT MAX(film.length) FROM film", con=connection)
max_len = df['max'][0]
print("Najdłuższy film we wszystkich kategoriach ma długość: {a}".format(a=max_len))

Najdłuższy film we wszystkich kategoriach ma długość: 185


#### Najdłuższy film ma długość 185, co zgadza się z Zadaniem 10, w którym to najdłuższe filmy również miały taką długość.

## ZADANIA IMPLEMENTACYJNE

### FUNKCJA 1 - wypisywanie wszystkich filmów z podanej kategorii.

In [157]:
main.film_in_category(15)

,title,languge,category
0,Aladdin Calendar,English,Sports
1,Anonymous Human,English,Sports
2,Artist Coldblooded,English,Sports
3,Bubble Grosse,English,Sports
4,California Birds,English,Sports
...,...,...,...
69,Tights Dawn,English,Sports
70,Tourist Pelican,English,Sports
71,Trading Pinocchio,English,Sports
72,Tuxedo Mile,English,Sports


### FUNKCJA 2 - liczba filmów w podanej kategorii.

In [158]:
main.number_films_in_category(12)

,category,count
0,Music,51


### FUNKCJA 3 - liczba filmów o długości zawartej w zadanym przedziale.

In [159]:
main.number_film_by_length(100, 120)

,length,count
0,100,12
1,101,7
2,102,11
3,103,9
4,104,6
5,105,6
6,106,6
7,107,10
8,108,5
9,109,7


### FUNKCJA 4 - wszyscy klienci z danego miasta.

In [160]:
main.client_from_city('Abu Dhabi')

,city,first_name,last_name
0,Abu Dhabi,Tom,Milner


### FUNKCJA 5 - średnia wartość wypożyczenia dla filmów o zadanej długości.

In [161]:
main.avg_amount_by_length(50)

,length,avg
0,50,3.810598


### FUNKCJA 6 - klienci którzy obejrzeli łącznie zadaną liczbę lub więcej godzin filmów.

In [162]:
main.client_by_sum_length(500)

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


### FUNKCJA 7 - statystyka długości filmów dla zadanej kategorii.

In [163]:
main.category_statistic_length('Sports')

,category,avg,sum,min,max
0,Sports,128.202703,9487,47,184
